In [ ]:
# Paso 1: Importar las librerías necesarias
import pandas as pd
import numpy as np
import folium
!pip install pulp
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, value

In [ ]:
# Paso 2: Cargar los datos del usuario
from google.colab import files
# Cargar la tabla de vehículos
print("Cargue el archivo con la tabla de vehículos (formato CSV: Placa, Capacidad):")
uploaded_vehicles = files.upload()
vehicles_df = pd.read_csv(list(uploaded_vehicles.keys())[0])
# Cargar la tabla de clientes
print("Cargue el archivo con los datos de ruteo (formato CSV: Cliente, Longitud, Latitud, Demanda):")
uploaded_routing = files.upload()
routing_df = pd.read_csv(list(uploaded_routing.keys())[0])

Cargue el archivo con la tabla de vehículos (formato CSV: Placa, Capacidad):


Saving Vehicles_Colombia.csv to Vehicles_Colombia (4).csv
Cargue el archivo con los datos de ruteo (formato CSV: Cliente, Longitud, Latitud, Demanda):


Saving Clients_Colombia_20.csv to Clients_Colombia_20 (2).csv


In [ ]:
# Paso 3: Mostrar los datos cargados
print("Datos de vehículos:")
print(vehicles_df)
print("Datos de ruteo:")
print(routing_df)

Datos de vehículos:
   Placa  Capacidad
0  Veh-1       1219
1  Veh-2       1000
2  Veh-3        961
Datos de ruteo:
       Cliente   Longitud   Latitud  Demanda
0   Cliente-28 -74.872615  4.024855      113
1   Cliente-16 -74.678434  4.160169      146
2   Cliente-24 -74.591933  4.558194      137
3   Cliente-18 -74.843408  4.065291      140
4    Cliente-9 -74.764999  4.106743      149
5   Cliente-10 -74.761998  4.824802      145
6   Cliente-29 -75.121692  4.669022      126
7   Cliente-25 -75.432672  4.188577      114
8   Cliente-13 -74.582121  4.441774      149
9    Cliente-1 -75.337866  4.002638      100
10   Cliente-5 -75.472847  4.087083      102
11  Cliente-17 -74.553730  4.555729      108
12   Cliente-6 -75.194854  4.330924      108
13  Cliente-14 -74.823328  4.154017      112
14  Cliente-12 -74.820476  4.365179      146
15  Cliente-23 -75.181856  4.681154      121
16   Cliente-2 -75.144288  4.534144      150
17   Cliente-3 -74.711670  4.363129      124
18  Cliente-26 -74.589090  4.

In [ ]:
# Instalar OR-Tools
!pip install ortools

# Importar las librerías necesarias
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
from scipy.spatial.distance import cdist

In [ ]:
# Solicitar el punto de origen
print("Ingrese las coordenadas del punto de origen:")
lat_origen = float(input("Latitud del origen: "))
long_origen = float(input("Longitud del origen: "))

# Agregar el punto de origen a la lista de ubicaciones
origin = {"Cliente": "Origen", "Longitud": long_origen, "Latitud": lat_origen, "Demanda": 0}
routing_df = pd.concat([pd.DataFrame([origin]), routing_df], ignore_index=True)

# Crear la matriz de distancias
locations = routing_df[['Longitud', 'Latitud']].values
dist_matrix = cdist(locations, locations, metric='euclidean')

# Crear el modelo de ruteo
def create_data_model():
    data = {}
    data['distance_matrix'] = dist_matrix
    data['demands'] = routing_df['Demanda'].values
    data['vehicle_capacities'] = vehicles_df['Capacidad'].values
    data['num_vehicles'] = len(vehicles_df)
    data['depot'] = 0  # Índice del punto de origen
    return data

data = create_data_model()

# Crear el administrador y el modelo de ruteo
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)

# Función de costo
def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return int(data['distance_matrix'][from_node][to_node] * 1000)  # Convertir a enteros para OR-Tools

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Restricciones de capacidad
def demand_callback(from_index):
    from_node = manager.IndexToNode(from_index)
    return data['demands'][from_node]

demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  # Sin capacidad extra
    data['vehicle_capacities'],  # Capacidad máxima por vehículo
    True,  # Comenzar en el depósito
    'Capacity'
)

# Parámetros de búsqueda
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

# Resolver el problema
solution = routing.SolveWithParameters(search_parameters)

# Mostrar resultados
if solution:
    print("Solución:")
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(node_index)
            index = solution.Value(routing.NextVar(index))
        route.append(manager.IndexToNode(index))
        print(f"Vehículo {vehicle_id + 1}: {route}")
else:
    print("No se encontró solución.")


Ingrese las coordenadas del punto de origen:
Latitud del origen: 4.188577
Longitud del origen: -74.553730
Solución:
Vehículo 1: [0, 14, 4, 1, 10, 11, 8, 13, 17, 15, 18, 0]
Vehículo 2: [0, 9, 12, 3, 19, 6, 20, 16, 7, 0]
Vehículo 3: [0, 2, 5, 0]


In [ ]:
# Crear un mapa de rutas por vehículo
vehicle_routes = {veh: [] for veh in vehicles_df['Placa']}

# Extraer las rutas calculadas con OR-Tools
for vehicle_id in range(data['num_vehicles']):
    index = routing.Start(vehicle_id)
    while not routing.IsEnd(index):
        node_index = manager.IndexToNode(index)
        vehicle_routes[vehicles_df['Placa'][vehicle_id]].append(node_index)
        index = solution.Value(routing.NextVar(index))

# Crear el mapa centrado en las ubicaciones promedio
m = folium.Map(location=[routing_df['Latitud'].mean(), routing_df['Longitud'].mean()], zoom_start=12)

# Añadir el origen al mapa
origin_row = routing_df.iloc[data['depot']]
folium.Marker(
    location=[origin_row['Latitud'], origin_row['Longitud']],
    popup="Origen (Depósito)",
    icon=folium.Icon(color="red", icon="home")
).add_to(m)

# Añadir los clientes al mapa
for index, row in routing_df.iterrows():
    if index != data['depot']:  # Evitar duplicar el origen
        folium.Marker(
            location=[row['Latitud'], row['Longitud']],
            popup=f"Cliente: {row['Cliente']} (Demanda: {row['Demanda']})",
            icon=folium.Icon(color="blue", icon="info-sign")
        ).add_to(m)

# Colores únicos para cada vehículo
import itertools
colors = itertools.cycle(['blue', 'green', 'red', 'purple', 'orange', 'darkblue', 'darkgreen', 'cadetblue'])

# Dibujar las rutas en el mapa con colores distintos para cada vehículo
for veh, route in vehicle_routes.items():
    color = next(colors)
    for i in range(len(route) - 1):
        # Dibujar las rutas entre los clientes
        folium.PolyLine(
            locations=[
                [routing_df.iloc[route[i]]['Latitud'], routing_df.iloc[route[i]]['Longitud']],
                [routing_df.iloc[route[i + 1]]['Latitud'], routing_df.iloc[route[i + 1]]['Longitud']]
            ],
            color=color,
            weight=2.5,
            opacity=1,
            tooltip=f"Vehículo: {veh}"
        ).add_to(m)
    # Dibujar la ruta desde el depósito hasta el primer cliente
    folium.PolyLine(
        locations=[
            [origin_row['Latitud'], origin_row['Longitud']],
            [routing_df.iloc[route[0]]['Latitud'], routing_df.iloc[route[0]]['Longitud']]
        ],
        color=color,
        weight=2.5,
        opacity=0.7,
        tooltip=f"Vehículo: {veh} (Salida desde origen)"
    ).add_to(m)
    # Dibujar la ruta desde el último cliente hasta el depósito
    folium.PolyLine(
        locations=[
            [routing_df.iloc[route[-1]]['Latitud'], routing_df.iloc[route[-1]]['Longitud']],
            [origin_row['Latitud'], origin_row['Longitud']]
        ],
        color=color,
        weight=2.5,
        opacity=0.7,
        tooltip=f"Vehículo: {veh} (Regreso al origen)"
    ).add_to(m)

# Mostrar el mapa
m

In [ ]:
# Generar el plan de ruteo
plan_ruteo = {vehiculo: [] for vehiculo in vehicles_df['Placa']}

# Extraer las rutas calculadas con OR-Tools
for vehicle_id in range(data['num_vehicles']):
    index = routing.Start(vehicle_id)
    while not routing.IsEnd(index):
        node_index = manager.IndexToNode(index)
        if node_index != data['depot']:  # Excluir el depósito
            plan_ruteo[vehicles_df['Placa'][vehicle_id]].append(routing_df.iloc[node_index]['Cliente'])
        index = solution.Value(routing.NextVar(index))

# Mostrar el plan de ruteo
print("Plan de Ruteo:")
for vehiculo, clientes in plan_ruteo.items():
    print(f"Vehículo: {vehiculo}")
    print(f"Rutas: {' -> '.join(clientes) if clientes else 'Sin asignación'}")
    print()

Plan de Ruteo:
Vehículo: Veh-1
Rutas: Cliente-14 -> Cliente-18 -> Cliente-28 -> Cliente-1 -> Cliente-5 -> Cliente-25 -> Cliente-6 -> Cliente-2 -> Cliente-12 -> Cliente-3

Vehículo: Veh-2
Rutas: Cliente-13 -> Cliente-17 -> Cliente-24 -> Cliente-26 -> Cliente-10 -> Cliente-4 -> Cliente-23 -> Cliente-29

Vehículo: Veh-3
Rutas: Cliente-16 -> Cliente-9



In [ ]:
# Inicializar el plan de ruteo con información de carga
plan_ruteo = {vehiculo: {"Clientes": [], "CargaTotal": 0} for vehiculo in vehicles_df['Placa']}

# Extraer las rutas calculadas con OR-Tools y calcular la carga
for vehicle_id in range(data['num_vehicles']):
    index = routing.Start(vehicle_id)
    while not routing.IsEnd(index):
        node_index = manager.IndexToNode(index)
        if node_index != data['depot']:  # Excluir el depósito
            cliente = routing_df.iloc[node_index]['Cliente']
            demanda = routing_df.iloc[node_index]['Demanda']
            plan_ruteo[vehicles_df['Placa'][vehicle_id]]["Clientes"].append(cliente)
            plan_ruteo[vehicles_df['Placa'][vehicle_id]]["CargaTotal"] += demanda
        index = solution.Value(routing.NextVar(index))

# Mostrar el plan de ruteo con la carga total de cada vehículo
print("Plan de Ruteo:")
for vehiculo, info in plan_ruteo.items():
    clientes = info["Clientes"]
    carga_total = info["CargaTotal"]
    print(f"Vehículo: {vehiculo}")
    print(f"Rutas: {' -> '.join(clientes) if clientes else 'Sin asignación'}")
    print(f"Carga Total: {carga_total}")
    print()


Plan de Ruteo:
Vehículo: Veh-1
Rutas: Cliente-14 -> Cliente-18 -> Cliente-28 -> Cliente-1 -> Cliente-5 -> Cliente-25 -> Cliente-6 -> Cliente-2 -> Cliente-12 -> Cliente-3
Carga Total: 1209

Vehículo: Veh-2
Rutas: Cliente-13 -> Cliente-17 -> Cliente-24 -> Cliente-26 -> Cliente-10 -> Cliente-4 -> Cliente-23 -> Cliente-29
Carga Total: 992

Vehículo: Veh-3
Rutas: Cliente-16 -> Cliente-9
Carga Total: 295

